In [ ]:
%run my_import.py
print(loaded_data.keys()) #tutti i dataset
series_dict = {}
refresh_data = False

## ***02_PREPROCESSING PHASE***

### **WHITE NOISE**

In [ ]:
#generate data
n_years = 10
start_date = "2014-01-01"
series_name = "White Noise"
date_range = pd.date_range(start=start_date, periods=365*n_years, freq='D')
np.random.seed(42)
white_noise = np.random.normal(loc=0, scale=1, size=len(date_range))
series = pd.Series(white_noise, index=date_range)
series_dict[series_name] = series

Questa serie è pronta così, non c'è bisogno di toccarla

### **ENG TEMP** (in pausa)

In [ ]:
engTemp = loaded_data['EngTemp']

In [ ]:
engTemp.set_index("date", inplace=True)
engTemp = engTemp.replace(-99.9, np.nan)
engTemp["mean temperature"] = engTemp.mean(axis=1)
engTemp.head()

In [ ]:
print("STATISTICHE DESCRITTIVE PER LA TEMPERATURA MEDIA",engTemp["mean temperature"].describe())
print("DIMENSIONE:",engTemp.shape)
print("NAN:",engTemp.isna().sum())
print("Giorni in cui la temperatura media è NaN:",engTemp["mean temperature"][engTemp["mean temperature"].isna()])

In [ ]:
#Ho solo 10 NaN su 60k righe, e sono del 1850. Posso eliminare quei record
#elimino le righe che hanno NaN nella colonna mean temperature
engTemp = engTemp.dropna(subset=["mean temperature"])

In [ ]:
print("DIMENSIONE:",engTemp.shape)
print("NAN:",engTemp.isna().sum())
print("Giorni in cui la temperatura media è NaN:",engTemp["mean temperature"][engTemp["mean temperature"].isna()])
#Ora non abbiamo più NaN nella colonna mean temperature

In [ ]:
#ACF E PACF
plot_acf(engTemp["mean temperature"], lags=40)
plot_pacf(engTemp["mean temperature"], lags=40)
plt.show()


In [ ]:
engTemp["mean temperature"].isna().sum()

In [ ]:
print(engTemp["mean temperature"].shape)
#prima un NaN era al 20/05/1853
engTemp["mean temperature"].loc['1853-05-19':'1853-05-21'].head()
# I NaN non ci sono più. Abbiamo skippato i giorni


In [ ]:
#plot zoommato solo del 2024
#plot con ts_plotly per store 1 e 50
series_dict = {
    'Temp zoommata': engTemp["mean temperature"].loc['2020-01-01':'2025-03-31']
    
}
ts_plotly(series_dict, title="Mid England Temperature in the last five years")

### **OIL PRICE**

In [ ]:
oil_prices = loaded_data['oil_prices']
oil_prices.reset_index(inplace=True)
oil_prices['date'] = oil_prices['date'].dt.strftime('%Y-%m')
oil_prices.set_index('date', inplace=True)
oil_prices.head()

In [ ]:
print("DIMENSIONE:",oil_prices.shape)
print("NAN:",oil_prices.isna().sum())

In [ ]:
print("STATISTICHE DESCRITTIVE:",oil_price.describe())

### **missing data**: serie univariata delle sales di Favorita

In [ ]:
missing = loaded_data['favorita_train']
missing.set_index("date", inplace=True)
missing = missing[['sales','family']]
missing.head()

In [ ]:
#sommo tutte le vendite di un singolo store per ogni giorno
missing = missing.groupby(['date']).sum().reset_index()
missing.drop(columns=["family"], inplace=True)
missing = missing.rename(columns={"sales": "total_daily_sales"})
missing.set_index("date", inplace=True)

In [ ]:
ts_plotly({"sales": missing['total_daily_sales']}, title="Sales")

In [ ]:
#ACF E PACF
plot_acf(missing['total_daily_sales'], lags=50)
plot_pacf(missing['total_daily_sales'], lags=50)
plt.show()

Alta ACF a lag 7, 14, 21 etc; ha senso.

In [ ]:
#introduciamo un 20% di NaN randomico con set_seed(24)
np.random.seed(24)
nan_indices = np.random.choice(missing.index, size=int(len(missing) * 0.2), replace=False)
missdata = missing.copy()
missdata.loc[nan_indices, 'total_daily_sales'] = np.nan
missdata.head()

In [ ]:
ts_plotly({"sales": missdata['total_daily_sales']}, title="Sales for store 1")

In [ ]:
print("Dimensione:",missdata.shape)
print("Quanti NAN ci sono?:",missdata.isna().sum())
print("Quanti valori buoni ci sono?:",missdata.notna().sum())
print("Percentuale di NAN:",missdata.isna().sum()/len(missdata)*100)


Quindi ho 1684 totali, di cui 1348 sono buoni e 336 NaN

### **outliers data**: serie univariata delle sales di Favorita

In [ ]:
outl = missing.copy()
print("Dimensione:",outl.shape)
print("Quanti NAN ci sono?:",outl.isna().sum())

In [ ]:
def create_outliers():
    # Creo qualche outlier in maniera randomica
    np.random.seed(22)
    # Genera 7 valori random tra 2,000,000 e 4,000,000
    outliers_positive = np.random.randint(2000000, 4000000, 7)
    # Genera 4 valori random tra -2,000,000 e -4,000,000
    outliers_negative = np.random.randint(-3000000, -1000000, 4)
    # Combina gli outlier positivi e negativi
    outliers = np.concatenate((outliers_positive, outliers_negative))
    return outliers

In [ ]:
# Crea gli outlier
outliers = create_outliers()
# Sostituisci 11 valori casuali nel DataFrame con gli outlier
np.random.seed(22)
random_indices = np.random.choice(outl.size, 11, replace=False)
flat_df = outl.values.flatten()
flat_df[random_indices] = outliers
out = pd.DataFrame(flat_df.reshape(outl.shape), columns=outl.columns, index = missdata.index)

In [ ]:
ts_plotly({"sales": out['total_daily_sales']}, title="Sales with outliers")
print("Dimensione:",out.shape)

In [ ]:
#ACF E PACF
plot_acf(outl['total_daily_sales'], lags=40)
plot_pacf(outl['total_daily_sales'], lags=40)
plt.show()

Ovviamente uguale a prima

### **SOLAR ENERGY**

In [ ]:
SolarData = loaded_data['solar_power']
SolarData.set_index('DATE_TIME',inplace=True)
SolarData.head()

In [ ]:
print("Quanti NAN ci sono?:",SolarData.isna().sum())
SolarData.shape
print(SolarData.corr())


aggiungo l'ora del giorno

In [ ]:
df = SolarData.copy()
df_reset = df.reset_index()
df_reset['DATE_TIME'] = pd.to_datetime(df_reset['DATE_TIME'])

# NUOVE VARIABILI TEMPORALI
df_reset['giorno_settimana'] = df_reset['DATE_TIME'].dt.dayofweek + 1
df_reset['ora_del_giorno'] = df_reset['DATE_TIME'].dt.hour + 1
#df_reset['settimana_del_mese'] = ((df_reset['DATE_TIME'].dt.day - 1) // 7) + 1
#df_reset['settimana_del_anno'] = df_reset['DATE_TIME'].dt.isocalendar().week
#df_reset['giorno_del_mese'] = df_reset['DATE_TIME'].dt.day
#df_reset['mese_del_anno'] = df_reset['DATE_TIME'].dt.month

df = df_reset.set_index('DATE_TIME')
df.head()

In [ ]:
#plot dela potenza DC vs ora del giorno
plt.figure(figsize=(12, 6))
sns.boxplot(data=df, x='ora_del_giorno', y='DC_POWER')
plt.title('DC Power vs Hour of the Day')
plt.xlabel('Hour of the Day')
plt.ylabel('DC Power')
plt.xticks(rotation=45)
plt.grid()
plt.show()

In [ ]:
print(df.shape)
print(df.info())

In [ ]:
plot_acf(df['DC_POWER'],lags=500)
plot_acf(df['DC_POWER'],lags=75)
plot_pacf(df['DC_POWER'],lags=20)
plt.show()

Grande ciclicità, tipico dei dati solari. Da qui vediamo le caratteristiche principali del dataset

### **AIR QUALITY** (in pausa)

In [ ]:
aqi = loaded_data['aqi']
aqi.drop(columns=["Unnamed: 15","Unnamed: 16"], inplace = True)

In [ ]:
print("Dimensione:",aqi.shape)
print("Quanti NaN ci sono?",aqi.isna().sum())
print("Giorni in cui abbiamo NaN:",aqi["Date"][aqi["Date"].isna()])
#voglio eliminare tutte le osservazioni NaN perchè sono tutte alla fine
aqi.dropna(inplace=True)
print("dimensioni:",aqi.shape)

Abbiamo i NaN, in tutte le variabili, negli ultimi 114 record; si possono eliminare

In [ ]:
# concateno Date e Time in un'unica nuova colonna DateTime
aqi['DateTime'] = pd.to_datetime(aqi['Date'] + ' ' + aqi['Time'], format='%d/%m/%Y %H.%M.%S')
# DateTime come indice
aqi.set_index('DateTime', inplace=True)

# converto colonne Date e Time in datetime
aqi['Date'] = pd.to_datetime(aqi['Date'], format='%d/%m/%Y')
aqi['Time'] = pd.to_datetime(aqi['Time'], format='%H.%M.%S').dt.time

# creo variabili ora del giorno e giorno della settimana
aqi['ora_del_giorno'] = pd.to_datetime(aqi['Time'], format='%H:%M:%S').dt.hour + 1
aqi['giorno_settimana'] = aqi['Date'].dt.dayofweek + 1

# droppo colonne Date e Time
aqi.drop(columns=["Date", "Time"], inplace=True)
aqi.index = pd.DatetimeIndex(pd.to_datetime(aqi.index))
aqi.info()

Ho alcune variabili object perchè contengono la virgola anzichè il punto

In [ ]:
#sostituisco le virgole con il punto e converto in float
aqi['CO(GT)'] = aqi['CO(GT)'].astype(str).str.replace(',', '.').astype(float)
aqi['C6H6(GT)'] = aqi['C6H6(GT)'].astype(str).str.replace(',', '.').astype(float)
aqi['T'] = aqi['T'].astype(str).str.replace(',', '.').astype(float)
aqi['RH'] = aqi['RH'].astype(str).str.replace(',', '.').astype(float)
aqi['AH'] = aqi['AH'].astype(str).str.replace(',', '.').astype(float)
aqi.info()

In [ ]:
ts_plotly({'CO':aqi['CO(GT)']}, title="Air Quality Index")
#vedo che i valori -200 sono segnati tali perchè sono NaN

In [ ]:
#vedo quanti valori di CO sono uguali a -200
print(aqi['CO(GT)'].value_counts().head())
#1683 valori della variabile target sono outlier
#nessuna di queste variabili può assumere valore negativo in natura ad eccezione della temperatura

In [ ]:
print(aqi['C6H6(GT)'].value_counts().head(3))
print(aqi['T'].value_counts().head(3))
print(aqi['RH'].value_counts().head(3))
print(aqi['AH'].value_counts().head(3))
#sembrano esserci 366 osservazioni comuni a molte variabili impostate come -200 (outlier)
#vedremo poi come gestirle

In [ ]:
#droppo una variabile con 90% NaN
print(aqi['NMHC(GT)'].value_counts().head())
#questa ha 8k valori outlier, non ha senso tenerla; la elimino
aqi.drop(columns=["NMHC(GT)"], inplace=True)
print("Variabile eliminata")

In [ ]:
#elimimo (mando a NaN) tutte le celle che contengono un valore minore di 100
#nessuna variabile può assumere valore minore di 100
aqi1 = aqi[aqi>-100]
aqi1.head()

In [ ]:
print(aqi1['CO(GT)'].value_counts().head(3))
print(aqi1['C6H6(GT)'].value_counts().head(3))
print(aqi1['T'].value_counts().head(3))
print(aqi1['RH'].value_counts().head(3))
print(aqi1['AH'].value_counts().head(3))
#ora non ci sono più i valori a -200 (outliers)

In [ ]:
aqi1.describe(include='all')

In [ ]:
#ts_plotly unico di NOX, NO2 e CO
ts_plotly({
    'NOX(GT)':aqi1['NOx(GT)'],
    'NO2':aqi1['NO2(GT)'],
    'CO':aqi1['CO(GT)']
           }, 
          title="Air Quality")

E' chiaro che ci sia un pattern in questi NaN. Sono quasi sovrapposti: Rispetto a NOX, la variabile CO ha 44 NaN in più, mentre NO2 solo 3 in più. Valuteremo poi come gestire questi NaN

In [ ]:
aqi1.isna().sum()

In [ ]:
# SUBPLOTS DELLE ALTRE VARIABILI
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Funzione ts_plotly modificata per plottare le variabili richieste
def ts_plotly(data_dict, title="Time Series"):
    fig = go.Figure()
    
    for key, series in data_dict.items():
        fig.add_trace(go.Scatter(
            x=series.index, 
            y=series,
            mode='lines',
            name=key
        ))
    
    fig.update_layout(
        title=title,
        xaxis_title="Data",
        yaxis_title="Valori",
        legend_title="Variabili",
        height=600,
        width=1000
    )
    
    return fig

# Selezionare solo le variabili con 366 valori (come indicato)
variables_to_plot = {
    'PT08.S1(CO)': aqi1['PT08.S1(CO)'],
    'C6H6(GT)': aqi1['C6H6(GT)'],
    'PT08.S2(NMHC)': aqi1['PT08.S2(NMHC)'],
    'PT08.S3(NOx)': aqi1['PT08.S3(NOx)'],
    'PT08.S4(NO2)': aqi1['PT08.S4(NO2)'],
    'PT08.S5(O3)': aqi1['PT08.S5(O3)'],
    'T': aqi1['T'],
    'RH': aqi1['RH'],
    'AH': aqi1['AH']
}
# Se preferisci vedere i grafici separatamente per chiarezza:
rows = 3
cols = 3
fig_multi = make_subplots(rows=rows, cols=cols, subplot_titles=list(variables_to_plot.keys()))

i = 0
for key, series in variables_to_plot.items():
    row = (i // cols) + 1
    col = (i % cols) + 1
    
    fig_multi.add_trace(
        go.Scatter(x=series.index, y=series, name=key),
        row=row, col=col
    )
    i += 1

fig_multi.update_layout(height=900, width=1000, title_text="Variabili di Qualità dell'Aria (366 valori)")
fig_multi.show()

In [ ]:
#Distribuzione delle potenziali variabili target
plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.hist(aqi1['CO(GT)'], bins=100, color='blue', alpha=0.7)
plt.title('CO(GT)')
plt.xlabel('CO(GT)')
plt.ylabel('Frequenza')
plt.subplot(1, 3, 2)
plt.hist(aqi1['C6H6(GT)'], bins=100, color='green', alpha=0.7)
plt.title('C6H6(GT)')
plt.xlabel('C6H6(GT)')
plt.ylabel('Frequenza')
plt.subplot(1, 3, 3)
plt.hist(aqi1['NOx(GT)'], bins=100, color='red', alpha=0.7)
plt.title('NOx(GT)')
plt.xlabel('NOx(GT)')
plt.ylabel('Frequenza')
plt.tight_layout()
plt.show()


In [ ]:
#matrice di correlazione heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(aqi1.corr(), annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Matrice di Correlazione')
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 4))
# Plot 1: Benzene vs Day of the Week
sns.boxplot(data=aqi1, x='giorno_settimana', y='C6H6(GT)', ax=axes[0])
axes[0].set_title('Benzene vs Day of the Week')
axes[0].set_xlabel('Day of the week')
axes[0].set_ylabel('Benzene')
axes[0].tick_params(axis='x', rotation=45)
axes[0].grid()
# Plot 2: Benzene vs Hour of the Day
sns.boxplot(data=aqi1, x='ora_del_giorno', y='C6H6(GT)', ax=axes[1])
axes[1].set_title('Benzene vs Hour of the Day')
axes[1].set_xlabel('Hour of the Day')
axes[1].set_ylabel('Benzene')
axes[1].tick_params(axis='x', rotation=45)
axes[1].grid()
plt.tight_layout()
plt.show()

Imputazione dati mancanti su tutto il dataset

In [ ]:
#IMPUTAZIONE
imputer = KNNImputer(n_neighbors=15)
aqi2 = pd.DataFrame(imputer.fit_transform(aqi1), columns=aqi1.columns)
aqi2.index = aqi1.index
#Plot after imputing
#ts_plotly unico di NOX, NO2 e CO
ts_plotly({
    'NOX(GT)':aqi2['NOx(GT)'],
    'NO2':aqi2['NO2(GT)'],
    'CO':aqi2['CO(GT)']
           }, 
          title="Series after KNN imputation")

Quindi aqi2 è il dataset pronto e pulito da mettere in pasto ai modelli

###### PCA; non sono sicuro si possa fare su time series data

In [ ]:
pca = PCA(n_components=4)
scaler = MinMaxScaler()
#TRY PCA
df_prova = aqi1.drop(columns=['C6H6(GT)']) #quella target la escludo
df_prova = df_prova.dropna()
df_scaled = scaler.fit_transform(df_prova)
principal_components = pca.fit_transform(df_scaled)
print("varianza spiegata dalle 3 componenti", pca.explained_variance_ratio_)
df_pca = pd.DataFrame(data=principal_components, index=pd.DataFrame(df_scaled).index, columns=['PC1','PC2','PC3','PC4'])

df_pca.index = df_prova.index
series_dict = {
    'PC1': df_pca['PC1'],
    'PC2': df_pca['PC2'],
    'PC3': df_pca['PC3'],
    'PC4': df_pca['PC4']
}
#ts_plotly(series_dict, title="PCA_AQI")



In [ ]:
principal_components1 = pca.fit(df_scaled)
loadings = principal_components1.components_
print("Loadings delle componenti principali:")
# Interpretazione dei coefficienti
for i, component in enumerate(loadings):
    print(f"COMPONENT {i+1}:")
    for var, coef in zip(df_prova.columns, component):
        print(f"{var}: {coef:.4f}")

### **F1 RACE DATA**

In [ ]:
f1 = loaded_data['f1']
f1.set_index("Date",inplace=True)
f1.head()

In [ ]:
drivers = f1["Driver"].unique()
for driver in drivers:
    f1_driver = f1[f1["Driver"]==driver]
    print(driver, len(f1_driver))
#Non hanno esattamente la stessa lunghezza
#Gasly, Ocon e Magnussen si sono ritirati durante la gara


Adesso il mio obiettivo è capire con quale PRECISA frequenza il sensore dell'autovettura rilascia i dati

In [ ]:
f1.reset_index(inplace=True)
f1['Date'] = pd.to_datetime(f1['Date'])
f1['freq'] = f1['Date'].shift(1)
f1['diff'] = f1['Date'] - f1['freq']

# Assumi che df sia il tuo DataFrame e 'DIFF' sia la colonna con i valori di differenza temporale
f1['diff_ms'] = f1['diff'].dt.total_seconds() * 1000
# Se vuoi arrotondare a millisecondi interi:
f1['diff_ms'][0] = 0
f1['diff_ms'] = f1['diff_ms'].round().astype(int)
f1.loc[f1['diff_ms'] < 0, 'diff_ms'] = 0 #metto a 0 differenze negative, non è possibile

In [ ]:
for driver in drivers:
    print(f"LA media delle rilevazioni per {driver} è:",
    f1[f1["Driver"] == driver]["diff_ms"].mean()),
    print(f"LA mediana delle rilevazioni per {driver} è:",
    f1[f1["Driver"] == driver]["diff_ms"].median())

Circa 130 millisecondi ogni record

In [ ]:
drivers_toplot = ['VER', 'HAM', 'LEC', 'NOR', 'GAS', 'ALB']
fig, axes = plt.subplots(2, 3, figsize=(8, 6))
for driver, ax in zip(drivers_toplot, axes.flatten()):
    driver_data = f1[f1["Driver"] == driver]
    ax.hist(driver_data["diff_ms"], bins=100, color='blue', alpha=0.7, edgecolor='black')
    ax.set_title(f"DIFF_MS per {driver}")
    ax.set_xlabel("diff_ms")
    ax.set_ylabel("Frequenza")
    ax.grid()
plt.tight_layout()
plt.show()

Tutte distribuzioni molto simili. Non so se questa differenza di rilevazione tra i record creerà problemi per i modelli

In [ ]:
#Distanza dal pilota davanti
series_dict = {
    'LEC': f1[f1["Driver"]=="LEC"]["DistanceToDriverAhead"],
    'NOR': f1[f1["Driver"]=="NOR"]["DistanceToDriverAhead"]}
ts_plotly(series_dict, title="Distance to Driver Ahead")

In [ ]:
#plot per vedere le serie storiche di RPM, questa è meglio
series_dict = {
    'LEC': f1[f1["Driver"]=="LEC"]["RPM"],
    'NOR': f1[f1["Driver"]=="NOR"]["RPM"]}
ts_plotly(series_dict, title="RPM")

In [ ]:
f1.info()

In [ ]:
f1_numeric = f1.drop(columns=['diff','diff_ms']).select_dtypes(include=[np.number,np.float64])
f1_numeric.corr()

In [ ]:
f1.sort_index(inplace=True)
f1.loc[f1['DistanceToDriverAhead'].isna()]

I NaN sulla variabile 'Distanza dal pilota davanti' ce li ha solamente il 1° in gara. Prima era NOR, poi è stato superato da VER e quindi da quel momento in poi VER ha sempre NaN su quella variabile.

In [ ]:
f1['DistanceToDriverAhead'][f1['Driver'] == 'LEC']

In [ ]:
plot_acf(f1['DistanceToDriverAhead'][f1['Driver'] == 'LEC'],lags=100)
plot_pacf(f1['DistanceToDriverAhead'][f1['Driver'] == 'LEC'], lags=100)
plt.show()

Ora voglio vedere per RPM (dobbiamo decidere quale variabile target)

In [ ]:
plot_acf(f1['RPM'][f1['Driver'] == 'LEC'],lags=150)
plot_pacf(f1['RPM'][f1['Driver'] == 'LEC'], lags=20)
plt.show()

### **ELECTRICTY DEMAND**

In [ ]:
elec = loaded_data['elec']
print("Dimensione:",elec.shape)
elec.head()

In [ ]:
def add_midnight_time(date_str):
    if len(date_str) < 19:  # Se la stringa è più corta di "2021-01-01 00:00:00"
        return date_str + " 00:00:00"
    return date_str

elec['Date'] = elec['Date'].apply(add_midnight_time)
elec['Date'] = pd.to_datetime(elec['Date'])
elec.set_index('Date',inplace=True)
elec.head()

In [ ]:
series_dict = {
    'Victoria (Australia) Electricity Demand': elec["Demand"]
}
ts_plotly(series_dict, title="Victoria (Australia) Electricity Demand")  

In [ ]:
series_dict = {
    'Victoria (Australia) Electricity Demand': elec.iloc[1500:2800]["Demand"]
}
ts_plotly(series_dict, title="Victoria (Australia) Electricity Demand in February 2014")  

Qui come regressori abbiamo la temperatura e la dummy del giorno festivo. Ora aggiungiamone altri

In [ ]:
#adding features
df_reset = elec.reset_index()
df_reset.rename(columns={'Date':'DATE_TIME'}, inplace=True)
df_reset['DATE_TIME'] = pd.to_datetime(df_reset['DATE_TIME'])

df_reset['giorno_settimana'] = df_reset['DATE_TIME'].dt.dayofweek + 1
df_reset['ora_del_giorno'] = df_reset['DATE_TIME'].dt.hour + 1
df_reset['settimana_del_mese'] = ((df_reset['DATE_TIME'].dt.day - 1) // 7) + 1
df_reset['settimana_del_anno'] = df_reset['DATE_TIME'].dt.isocalendar().week
df_reset['giorno_del_mese'] = df_reset['DATE_TIME'].dt.day
df_reset['mese_del_anno'] = df_reset['DATE_TIME'].dt.month
df_elec = df_reset.set_index('DATE_TIME')
df_elec.head()

In [ ]:
df_elec.corr().round(3)

In [ ]:
# Crea una figura con 3 subplot per i boxplot
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Boxplot per Category1
df_elec.boxplot(column='Demand', by='giorno_settimana', ax=axes[0])
axes[0].set_xlabel('Giorno della settimana')
axes[0].set_ylabel('Domanda di elettricità')

# Boxplot per Category2
df_elec.boxplot(column='Demand', by='ora_del_giorno', ax=axes[1])
axes[1].set_xlabel('Ora del giorno')
axes[1].set_ylabel('Domanda di elettricità')

# Boxplot per Category3
df_elec.boxplot(column='Demand', by='mese_del_anno', ax=axes[2])
axes[2].set_xlabel("Mese dell'anno")
axes[2].set_ylabel('Domanda di elettricità')

# Aggiungi spazio tra i subplot
plt.tight_layout()
plt.suptitle('')  
plt.show()

In [ ]:
plot_acf(df_elec['Demand'],lags=200)
plot_pacf(df_elec['Demand'], lags=150)
plt.show()

Grande ciclità come ci aspettavamo. La PACF mostra alte correlazioni nei lag (oltre a 1 e 2) 47,48,49, cioè il giorno dopo alla stessa ora (dati half-hour).

### **FAVORITA**

In [ ]:
favorita_train = loaded_data['favorita_train']
favorita_train.set_index("date", inplace=True)
favorita_train.drop(columns=['id'],inplace=True)
favorita_train.rename(columns={"store_nbr": "Store","family":"Product_type"}, inplace=True)
favorita_test = loaded_data['favorita_test']
favorita_test.set_index("date", inplace=True)
favorita_test.drop(columns=['id'],inplace=True)
favorita_test.rename(columns={"store_nbr": "Store","family":"Product_type"}, inplace=True)
print(favorita_train.shape)
favorita_train.head()

'onpromotion' ci dice quanti prodotti di quel tipo di prodotto erano in promozione in un quel giorno e in quel negozio

In [ ]:
print("Dimensioni del test",favorita_test.shape)
favorita_test.head(2)

In [ ]:
print("Quanti stores ci sono?",favorita_train["Store"].unique().shape[0]) #sono 54 stores
print("Quanti tipi di prodotto diverso ci sono?",favorita_train["Product_type"].unique().shape[0]) #ci sono 33 tipi di prodotto
print("Quanti giorni abbiamo?",favorita_train.index.unique().shape[0]) #ci sono 1684 giorni

In [ ]:
print("Top 5 store in termini di sales totali",favorita_train[["Store", "sales"]].groupby(["Store"]).mean().sort_values(by="sales", ascending=False).head())
print("Top 5 tipi di prodotto in termini di sales totali",favorita_train[["Product_type", "sales"]].groupby(["Product_type"]).mean().sort_values(by="sales", ascending=False).head())

In [ ]:
#Creo una dummy per le promozione
favorita_train['promo'] = favorita_train['onpromotion'].apply(lambda x: 0 if x == 0 else 1)
favorita_test['promo'] = favorita_test['onpromotion'].apply(lambda x: 0 if x == 0 else 1)

In [ ]:
#boxplot sales vs promo
plt.figure(figsize=(10,6))
sns.boxplot(x=favorita_train["promo"], 
            y=np.log1p(favorita_train["sales"]))  # Log transform for better visibility
plt.title("Promotion Impact on Sales (Log Scale)")
plt.xlabel("Item on Promotion")
plt.ylabel("Log(Sales + 1)")
plt.xticks([0,1], ["No Promotion", "Promoted"])
plt.show()

In [ ]:
#voglio aggiungere il prezzo del petrolio come regressore
oil = loaded_data['daily_oil']
oil.fillna(0,inplace=True)
oil.head()

In [ ]:
favorita_train = pd.merge(favorita_train, oil, on='date',how='left')
favorita_test = pd.merge(favorita_test, oil, on='date',how='left')

In [ ]:
favorita_train.rename(columns={"dcoilwtico":"oil price"},inplace=True)
favorita_test.rename(columns={"dcoilwtico":"oil price"},inplace=True)

Adding more temporal features

In [ ]:
# on train set
df_reset = favorita_train.reset_index()
df_reset.rename(columns={'date':'DATE_TIME'}, inplace=True)
df_reset['DATE_TIME'] = pd.to_datetime(df_reset['DATE_TIME'])

df_reset['giorno_settimana'] = df_reset['DATE_TIME'].dt.dayofweek + 1
df_reset['settimana_del_mese'] = ((df_reset['DATE_TIME'].dt.day - 1) // 7) + 1
df_reset['settimana_del_anno'] = df_reset['DATE_TIME'].dt.isocalendar().week
df_reset['giorno_del_mese'] = df_reset['DATE_TIME'].dt.day
df_reset['mese_del_anno'] = df_reset['DATE_TIME'].dt.month
favorita_train = df_reset.set_index('DATE_TIME')

#now on test set
df_reset = favorita_test.reset_index()
df_reset.rename(columns={'date':'DATE_TIME'}, inplace=True)
df_reset['DATE_TIME'] = pd.to_datetime(df_reset['DATE_TIME'])

df_reset['giorno_settimana'] = df_reset['DATE_TIME'].dt.dayofweek + 1
df_reset['settimana_del_mese'] = ((df_reset['DATE_TIME'].dt.day - 1) // 7) + 1
df_reset['settimana_del_anno'] = df_reset['DATE_TIME'].dt.isocalendar().week
df_reset['giorno_del_mese'] = df_reset['DATE_TIME'].dt.day
df_reset['mese_del_anno'] = df_reset['DATE_TIME'].dt.month
favorita_test = df_reset.set_index('DATE_TIME')

In [ ]:
favorita_test.head(2)

In [ ]:
#boxplot sales by day of week
dow_sales = favorita_train.groupby("giorno_settimana")["sales"].mean().reset_index()
sns.barplot(x="giorno_settimana", y="sales", data=dow_sales)
plt.title("Average Sales by Day of Week")
plt.xticks(rotation=45)
plt.show()

In [ ]:
favorita_train.isna().sum()

In [ ]:
nan_days = favorita_train[favorita_train['oil price'].isna()]['giorno_settimana'].value_counts()
print(nan_days)

I NaN che avevamo in oil price sono nel weekend nel quale il mercato è chiuso.

Adding features like 30 day rolling mean and lags

In [ ]:
favorita_train['rolling_mean_30'] = favorita_train.groupby(['Store',
                                            'Product_type'])['sales'].transform(lambda x: x.rolling(30, min_periods=1).mean())
favorita_train.reset_index(inplace=True)
favorita_train = favorita_train.sort_values(['Store','Product_type','DATE_TIME'])
for lag in [1, 2, 3, 7, 14, 28]:
    favorita_train[f'sales_lag_{lag}'] = favorita_train.groupby(['Store', 'Product_type'])['sales'].shift(lag)
favorita_train.set_index('DATE_TIME',inplace=True)
favorita_train.head()

### **ROSSMAN SALES** (in pausa)

In [ ]:
rossman_train = loaded_data['rossman_train']
rossman_test = loaded_data['rossman_test']
rossman_store = loaded_data['rossman_store']
rossman_train = rossman_train.sort_index()

In [ ]:
rossman_train.head()

Adding features

In [ ]:
rossman_train['rolling_mean_30'] = rossman_train.groupby('Store')['Sales'].transform(lambda x: x.rolling(30, min_periods=1).mean())
#how we do on test set, make sense?
rossman_train.reset_index(inplace=True)
rossman_train = rossman_train.sort_values(['Store','Date'])
for lag in [1, 2, 3, 7, 14, 28]:
    rossman_train[f'sales_lag_{lag}'] = rossman_train.groupby('Store')['Sales'].shift(lag)
rossman_train.set_index('Date',inplace=True)
rossman_train.head()
#should make sense also on test set?

In [ ]:
# Extracting year, month, day and week, and making new columns
rossman_train['Year'] = rossman_train.index.year
rossman_train['Month'] = rossman_train.index.month
rossman_train['Day'] = rossman_train.index.day
rossman_train['WeekOfYear'] = rossman_train.index.isocalendar().week
rossman_train['SalePerCustomer'] = rossman_train['Sales']/rossman_train['Customers'] 

In [ ]:
rossman_train.isnull().sum()

In [ ]:
nan_sales_customers = rossman_train[rossman_train['SalePerCustomer'].isna()][['Open','Sales', 'Customers']]
nan_sales_customers.head()

I NaN nel SalesperCustomer ci sono solo quando quello store in quella data ha venduto 0 (era chiuso). I NaN per i lags sono ovvi. Metto tutto a 0.

In [ ]:
rossman_train[['SalePerCustomer','rolling_mean_30','sales_lag_1','sales_lag_2','sales_lag_3','sales_lag_7','sales_lag_14','sales_lag_28']] = rossman_train[['SalePerCustomer','rolling_mean_30','sales_lag_1','sales_lag_2','sales_lag_3','sales_lag_7','sales_lag_14','sales_lag_28']].fillna(0)
rossman_train.isna().sum()

In [ ]:
print("Quanti giorni abbiamo?",rossman_train.index.unique().shape[0]) #942 giorni unici, circa 3 anni. Dal 1 Gennaio 2013 al 31 Luglio 2015 
print("Quanti store abbiamo?",rossman_train['Store'].unique().shape[0]) #1115 store diversi
print("Quanti giorni abbiamo di chiusura?",rossman_train[(rossman_train.Open == 0)].shape[0])
print("Dimensione train:",rossman_train.shape)
print("Quanti giorno il negozio era aperto ma hanno venduto 0?",rossman_train[(rossman_train.Open == 1) & (rossman_train.Sales == 0)].shape[0]) #54

Alcune combinazioni di store & data, sono mancanti, chiusure del negozio. A volte il negozio era chiuso ma comunque si è venduto zero (riassortimento/inventario)

In [ ]:
# Checking data when the stores were closed
train_store_closed = rossman_train[(rossman_train.Open == 0)]
train_store_closed.hist('DayOfWeek') #Quasi sempre la domenica

In [ ]:
#Vediamo il df store
rossman_store.isnull().sum()

In [ ]:
# Replacing missing values for CompetitonDistance with median, non ha senso sia 0 o NaN
rossman_store['CompetitionDistance'].fillna(rossman_store['CompetitionDistance'].median(), inplace=True)
rossman_store.fillna(0, inplace=True)

In [ ]:
# Joining the tables
index = rossman_train.index
train = pd.merge(rossman_train, rossman_store, on='Store', how='inner')
train.index = index
train.head()

store type sono 4 tipi di store differenti. 
assortment descrive un livello di assortimento più o meno esteso.

In [ ]:
train['StoreType'].value_counts()

In [ ]:
#Droppo le ultime 4 variabili, poco rilevanti secondo me (fonte Kaggle)
train = train.drop(columns=["CompetitionOpenSinceYear", "Promo2",
       'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval'])

In [ ]:
sns.catplot(data=train, x="Month", y="Sales", 
            col='Promo', hue='StoreType', row="Year", 
            kind='point', aspect=1.5)

plt.show()

In [ ]:
#Creo dummy per i giorni festivi a prescindere dal tipo di festività
train['is_holiday'] = ((train['StateHoliday'].isin(['a', 'b', 'c'])) | (train['SchoolHoliday'] == 1)).astype('category')
train = train.drop(columns=['StateHoliday', 'SchoolHoliday'])

In [ ]:
train['WeekOfYear']=train['WeekOfYear'].astype(int)
train['StoreType']=train['StoreType'].astype('category')
train['Assortment']=train['Assortment'].astype('category')

In [ ]:
train.drop(columns=['StoreType','Assortment','is_holiday']).corr().round(2)

In [ ]:
train.describe(include='all')

In [ ]:
plot_acf(train['Sales'][train['Store']==1],lags=50)
plot_pacf(train['Sales'][train['Store']==1],lags=50)
plt.show()

Alte correlazioni ai lag 7,14,21,28,35 etc...

##### TREATING TEST SET

In [ ]:
#il test va dal 1 agosto al 17 settembre del 2015 per tutti i 1115 stores

In [ ]:
rossman_test['Date'] = pd.to_datetime(rossman_test['Date'])
rossman_test.set_index('Date',inplace=True)
rossman_test = rossman_test.sort_index()
rossman_test.drop(columns=['Id'],inplace=True)

In [ ]:
# Assuming stores open in test
rossman_test.fillna(1, inplace=True)

In [ ]:
#adding also here temporal features
rossman_test['Month'] = rossman_test.index.month
rossman_test['Day'] = rossman_test.index.day
rossman_test['WeekOfYear'] = rossman_test.index.isocalendar().week
rossman_test['Year'] = rossman_test.index.year


In [ ]:
rossman_test['is_holiday'] = ((rossman_test['StateHoliday']=='a') | (rossman_test['SchoolHoliday'] == 1)).astype('category')
rossman_test = rossman_test.drop(columns=['StateHoliday', 'SchoolHoliday'])

In [ ]:
index = rossman_test.index
test = pd.merge(rossman_test, rossman_store, on='Store', how='inner')
test.index = index
#Droppo le ultime 4 variabili, poco rilevanti secondo me (fonte Kaggle)
test = test.drop(columns=["CompetitionOpenSinceYear", "Promo2",
       'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval'])

In [ ]:
print("Variabili del training set:",train.columns)
print("Variabili del test set:",test.columns) 

In [ ]:
set(train.columns) - set(test.columns) #variabili che sono nel train ma non nel test

Solo la variabile target, Customers, SaleperCustomers, rollingMean e le variabili Lags non ci sono nel test set (sono future, non possiamo saperle ora)

In [ ]:
test['Open']=test['Open'].astype(int)
test['WeekOfYear']=test['WeekOfYear'].astype(int)
test['StoreType']=test['StoreType'].astype('category')
test['Assortment']=test['Assortment'].astype('category')

In [ ]:
test.isna().sum()

In [ ]:
test.info()

In [ ]:
train.info()

Dati pronti per modelli